The **Part 1** of the project is divided into three sections:

1- Feature Extraction (Using SIFT)

2- Outlier Removal (Using RANSAC)

3- Computing the Homographies (Using DLT)


**pip install opencv-python**

**pip install opencv-contrib-python**

**Section 1**

In [1]:
#Imports
import numpy as np
import cv2
import os

In [2]:
#Display the video
capture = cv2.VideoCapture(os.path.abspath('trymefirst_lisbon.mp4'))
while(capture.isOpened()):
    ret, frame = capture.read()
    cv2.imshow('frame',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):  ##press q if you want the video to stop 
        break
capture.release()
cv2.destroyAllWindows()

In [ ]:
capture = cv2.VideoCapture(os.path.abspath('trymefirst_lisbon.mp4'))

sift_points = [] #nome a definir no config
t = 0 
sift = cv2.SIFT_create(100) #number of sift points
while True:
    t = t + 1
    if t == 5: break 
    success, frame = capture.read()
    if success:
        frame_points = []
        gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
        kp, des = sift.detectAndCompute(gray,None)
        frame_points = ([kp[0].pt[0],kp[0].pt[1]]+des[0].tolist())
        for i in range(1,len(kp)):
             list = ([kp[i].pt[0],kp[i].pt[1]]+des[i].tolist())
             frame_points = np.column_stack((frame_points,list))  
    sift_points.append(frame_points) #append everything into a list 

In [115]:
print((sift_points[0])[0,:])

[  5.67994452 177.10325623  42.47234726  42.47234726 268.80291748
 177.10325623 177.10325623 592.91009521 406.83502197 346.60913086
 329.41320801 329.41320801 165.71827698  62.68400955 406.83502197
  39.95887375 306.94552612 217.01228333  61.66337967 306.94552612
 117.35230255 331.00610352 425.39553833 751.51934814  61.66337967
 600.52026367 425.39553833  48.0570755  117.35230255  62.68400955
 130.28042603 751.51934814  37.95763779 515.91253662 515.91253662
 751.51934814 363.3117981   37.95763779 515.91253662  51.12105942
 340.10839844 169.31726074 321.1583252  520.44488525 207.70555115
 340.10839844 207.70555115 268.80291748 321.1583252  169.31726074
 346.60913086  20.52986145  71.78164673 645.96630859  70.93877411
 645.96630859  70.93877411 645.96630859 425.58969116 425.58969116
  44.46161652 149.44848633 211.03762817  88.5995636  211.03762817
 204.95632935  44.46161652 583.16009521 313.05712891 209.08547974
 204.95632935 313.05712891 204.95632935  64.32099152 209.08547974
 207.66563

In [ ]:
print((sift_points[0])[0,:])